# AAO GALAH RESEARCH JAN-APR 2016

### Missy McIntosh, Gayandhi de Silva, Jeffrey Simpson
http://www.univie.ac.at/webda/cgi-bin/frame_data_list.cgi?ascc111+ad2k+ad2000.coo

#### Check if there are any known open clusters in GALAH
#### This is a very slow python script....

In [1]:
import numpy as np
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.coordinates import FK4
import time
from tqdm import *




# Read in 2Mass input database Name, RA, and DEC information
# note this data is presorted by RA

twomass_data = np.genfromtxt('sortedgal.tbl', delimiter='\t', names=True, dtype=float)

twomass_ident = []
twomass_ra = []
twomass_dec = []
for i in tqdm(range(len(twomass_data))):
    twomass_ident.append(twomass_data[i][0])
    twomass_ra.append(twomass_data[i][1])
    twomass_dec.append(twomass_data[i][2])
    time.sleep(3)

/Users/mmcintosh/anaconda/lib/python3.4/site-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated. You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)


In [2]:
# Read in DIAS Name, RA, DEC
# I didn't know an easy way to read in this file so this is very messy and hacked and I apologize


dias_data = np.genfromtxt('dias.txt', delimiter='\t', names=None, dtype=None)
dias_data = dias_data[1:]


data = []
name = []
subsetra = []
subsetdec = []
for line in dias_data:
 
    line = line.decode('UTF-8')

    line = (str(line).split("  "))

    line = np.array(list(filter(None, line)))
    name.append(line[0])
    try:
        c = SkyCoord(line[1], line[2], unit=(u.hourangle, u.deg))
        subsetra.append(c.ra.degree)
        subsetdec.append(c.dec.degree)
    except:
        print("error: ", line)
    data.append(line)
    
# some problems with dec
subsetra = np.asarray(subsetra)
subsetdec = np.asarray(subsetdec)

In [75]:
print(len(twomass_ident))



8395154


In [3]:
# Read in Webda observed Name, RA, and DEC information
webda_data = np.genfromtxt('ad2000.coo', delimiter='\t', names=None, dtype=None)

#subset = [[10.5680911, 109.8102128, 27.831013, 1.1438403, 276.1315219, 1, 400],[-89.969946899999997, -89.9304625, 5,5,-89.915107199999994,5,-89.915107199999994]]
subsetra = []
subsetdec = []
for i in np.arange(2,len(webda_data)):
    coords = [(webda_data[i][2].decode('UTF-8'))+" "+ (webda_data[i][3].decode('UTF-8'))]
    c = SkyCoord(coords, unit=(u.hourangle, u.deg))
    subsetra.append(c.ra.degree[0])
    subsetdec.append(c.dec.degree[0])
    
#subsetra.extend(subset[0])
#subsetdec.extend(subset[1])
subsetra = np.asarray(subsetra)
subsetdec = np.asarray(subsetdec)


NameError: name 'webda_data' is not defined

In [11]:
def pbsearch(sorted_data, subset, tolerance):
    tol = tolerance

    # return index if it exists, otherwise, print "not found"
    searched_arr = []
    matched_arr = []
    bar = progressbar.ProgressBar(maxval=len(subset), \
    widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
    bar.start()
    for i in tqdm(range(len(subset))):
        a = subset[i]
        time.sleep(3)
        ra = sorted_data
        n = len(ra)
        for element in np.arange(0,n):
            bar.update(element+1)
            sleep(0.1)
            n = len(ra)
            if a - tol > ra[int(n/2)]:
                ra = ra[int(n/2):n]
            elif a + tol < ra[int(n/2)]:
                ra = ra[0:int(n/2)]
            else:
                match = ra[int(n/2)]
                #print('match!  searched', a, " found", match, " which is", np.absolute(a - ra[int(n/2)]), "away")
                matched_arr.append(match)
                searched_arr.append(a)
                break
                
            if n > 1:
                continue
            else:
                break
    bar.finish() 
    print("searched for a subset of ", len(subset), "RAs against 2mass list of", len(sorted_data), "RAs")
    print("RA matches: ", len(matched_arr), "with a tolerance of ", tol)
    return(searched_arr,matched_arr)

In [ ]:
# return subset RAs found within sorted data
tol = 0.0005
searched_ra, matched_ra = pbsearch(twomass_ra, subsetra, tol)

  1%|▏         | 32/2168 [02:46<3:06:46,  5.25s/it]

In [41]:
# retrieve the DEC and indicies of the found subset RAs
twomassind_arr = []
subsetind_arr = []
twomassdec_arr = []
subsetdec_arr = []

for element in matched_ra:
    twomassind = np.where(ra == element)[0][0]
    twomassind_arr.append(twomassind)
    matcheddec = dec[twomassind]
    twomassdec_arr.append(matcheddec)
    
for element in searched_ra:    
    subsetind = np.where(subsetra == element)[0][0]
    subsetind_arr.append(subsetind)
    searcheddec = subsetdec[subsetind]
    subsetdec_arr.append(searcheddec)
    
#print(twomassind_arr, twomassdec_arr)
#print(subsetind_arr, subsetdec_arr)

IndexError: index 0 is out of bounds for axis 0 with size 0

In [48]:
print(matched_ra)

#np.where(ra == element)[0][0]


0.0876575


In [ ]:
# check that the DEC values are w/i tolerence and return matched indices
# indices returned are for 2MASS
matched_ind = []
for i in np.arange(0,len(twomassdec_arr)):
    a = subsetdec_arr[i]
    dec = twomassdec_arr[i]
    if np.absolute(a - dec) < tol:
        matched_ind.append(ind_arr[i])
        
print("searched for a subset of ", len(subsetra), "Coordinates against 2mass list of", len(twomass_ra), "Coordinates")
print("Coordinate matches: ", len(matched_ra))        
print("IDs: ", matched_ind)

In [ ]:
# select info on the matched RA and DECs
twomass_data['galah_id'][matched_ind]

In [10]:
import time
from tqdm import *